In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import glob
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D,Dropout,Concatenate,GlobalAveragePooling2D,AveragePooling2D, MaxPooling2D, Flatten, Dense,BatchNormalization,Add,Input,Activation

In [5]:
!pip install opencv-python

# Importing data and train test split

In [7]:
import glob
import librosa
import numpy as np
import cv2 
Mild_Demented = glob.glob('/kaggle/input/alzheimer-mri-dataset/Dataset/Mild_Demented/*.jpg')
Moderate_Demented = glob.glob('/kaggle/input/alzheimer-mri-dataset/Dataset/Moderate_Demented/*.jpg')
Non_Demented= glob.glob('/kaggle/input/alzheimer-mri-dataset/Dataset/Non_Demented/*.jpg')
Very_Mild_Demented= glob.glob('/kaggle/input/alzheimer-mri-dataset/Dataset/Very_Mild_Demented/*.jpg')

data = []
labels = []

image_size=(128,128)

for i in Mild_Demented:   
    image = cv2.imread(i)
    image = cv2.resize(image, image_size)
    
    data.append(image)
    labels.append(0)
    
for i in Moderate_Demented:   
    image = cv2.imread(i)
    image = cv2.resize(image, image_size)
    
    data.append(image)
    labels.append(1)
    
for i in Non_Demented:   
    image = cv2.imread(i)
    image = cv2.resize(image, image_size)
    
    data.append(image)
    labels.append(2)
    
for i in Very_Mild_Demented:   
    image = cv2.imread(i)
    image = cv2.resize(image, image_size)
    
    data.append(image)
    labels.append(3)

X=np.array(data)
y=np.array(labels)
print(X.shape)
print(y.shape)

(6400, 128, 128, 3)
(6400,)


In [8]:
# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Perform train-validation split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(4096, 128, 128, 3)
(1024, 128, 128, 3)
(1280, 128, 128, 3)
(4096,)
(1024,)
(1280,)


In [10]:
# Transform targets to keras compatible format
#y_train = np_utils.to_categorical(y_train, num_classes=4)
#y_val = np_utils.to_categorical(y_val, num_classes=4)
#y_test = np_utils.to_categorical(y_test, num_classes=4)

#Normalize data
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')
 
X_train = X_train / 255.0
X_val = X_val/ 255.0
X_test = X_test/ 255.0

In [20]:
y_train


array([0, 3, 0, ..., 3, 2, 0])

# GoogleNet Architecture Construction

In [24]:
#Creating the inception module

def inception_module(x,
                     filter_1x1_number,
                     filter_1x1_before_3x3_number,
                     filter_3x3_number,
                     filter_1x1_before_5x5_number,
                     filter_5x5_number,
                     max_pool_filter_number,
                     name
                     ):
    
  
    conv_1x1 = Conv2D(filter_1x1_number, (1, 1),padding='same',activation='relu')(x)

    conv_3x3 = Conv2D(filter_1x1_before_3x3_number, (1, 1),padding='same',activation='relu')(x)
    conv_3x3 = Conv2D(filter_3x3_number, (3, 3),padding='same',activation='relu')(conv_3x3)

    conv_5x5 = Conv2D(filter_1x1_before_5x5_number, (1, 1),padding='same',activation='relu')(x)
    conv_5x5 = Conv2D(filter_5x5_number, (5, 5),padding='same',activation='relu')(conv_5x5)

    max_pool = MaxPooling2D(pool_size=(3, 3),strides=(1,1),padding='same')(x)
    max_pool = Conv2D(max_pool_filter_number, (1, 1),padding='same',activation='relu')(max_pool)

    output =Concatenate(axis=3, name=name)([conv_1x1,conv_3x3,conv_5x5,max_pool])

    return output

In [25]:
image_shape=(128, 128, 3)
classes=4

inputs = Input(shape=image_shape)

#Step 1 Initial convolution and maxpool layers before inception modules

#7x7 conv followed by maxpool
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2')(inputs)
x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_1_3x3/2')(x)

#3x3 conv(1x1 reduced) followed by maxpool
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv_2a_3x3/1')(x)
x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2b_3x3/1')(x)

x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_2_3x3/2')(x)


#Step 2 Inception modules one after another
#Inception module 3(a), 3(b) followed by maxpool
x = inception_module(x,
                     filter_1x1_number=64,
                     filter_1x1_before_3x3_number=96,
                     filter_3x3_number=128,
                     filter_1x1_before_5x5_number=16,
                     filter_5x5_number=32,
                     max_pool_filter_number=32,
                     name='inception_3a'
                     )

x = inception_module(x,
                     filter_1x1_number=128,
                     filter_1x1_before_3x3_number=128,
                     filter_3x3_number=192,
                     filter_1x1_before_5x5_number=32,
                     filter_5x5_number=96,
                     max_pool_filter_number=64,
                     name='inception_3b'
                     )

x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_3_3x3/2')(x)

#Inception module 4(a) to 4(e) followed by maxpool [auxilliary branch after 4(a) and 4(d)]
x = inception_module(x,
                     filter_1x1_number=192,
                     filter_1x1_before_3x3_number=96,
                     filter_3x3_number=208,
                     filter_1x1_before_5x5_number=16,
                     filter_5x5_number=48,
                     max_pool_filter_number=64,
                     name='inception_4a'
                     )

#auxilliary branch after 4(a)
aux_1 = AveragePooling2D((5, 5), strides=3)(x)
aux_1 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux_1)

aux_1 = Flatten()(aux_1)

aux_1 = Dense(1024, activation='relu')(aux_1)
aux_1 = Dropout(0.7)(aux_1)
aux_1 = Dense(classes, activation='softmax', name='auxilliary_output_1')(aux_1)


x = inception_module(x,
                     filter_1x1_number=160,
                     filter_1x1_before_3x3_number=112,
                     filter_3x3_number=224,
                     filter_1x1_before_5x5_number=24,
                     filter_5x5_number=64,
                     max_pool_filter_number=64,
                     name='inception_4b'
                     )

x = inception_module(x,
                     filter_1x1_number=128,
                     filter_1x1_before_3x3_number=128,
                     filter_3x3_number=256,
                     filter_1x1_before_5x5_number=24,
                     filter_5x5_number=64,
                     max_pool_filter_number=64,
                     name='inception_4c'
                     )

x = inception_module(x,
                     filter_1x1_number=112,
                     filter_1x1_before_3x3_number=144,
                     filter_3x3_number=288,
                     filter_1x1_before_5x5_number=32,
                     filter_5x5_number=64,
                     max_pool_filter_number=64,
                     name='inception_4d'
                     )

#auxilliary branch after 4(d)
aux_2 = AveragePooling2D((5, 5), strides=3)(x)
aux_2 = Conv2D(128, (1, 1), padding='same', activation='relu')(aux_2)

aux_2 = Flatten()(aux_2)

aux_2 = Dense(1024, activation='relu')(aux_2)
aux_2 = Dropout(0.7)(aux_2)
aux_2 = Dense(classes, activation='softmax', name='auxilliary_output_2')(aux_2)

x = inception_module(x,
                     filter_1x1_number=256,
                     filter_1x1_before_3x3_number=160,
                     filter_3x3_number=320,
                     filter_1x1_before_5x5_number=32,
                     filter_5x5_number=128,
                     max_pool_filter_number=128,
                     name='inception_4e'
                     )

x = MaxPooling2D((3, 3), padding='same', strides=(2, 2), name='max_pool_4_3x3/2')(x)


#Inception module 5(a) and 5(b) followed by avearge pooling

x = inception_module(x,
                     filter_1x1_number=256,
                     filter_1x1_before_3x3_number=160,
                     filter_3x3_number=320,
                     filter_1x1_before_5x5_number=32,
                     filter_5x5_number=128,
                     max_pool_filter_number=128,
                     name='inception_5a'
                     )

x = inception_module(x,
                     filter_1x1_number=384,
                     filter_1x1_before_3x3_number=192,
                     filter_3x3_number=384,
                     filter_1x1_before_5x5_number=48,
                     filter_5x5_number=128,
                     max_pool_filter_number=128,
                     name='inception_5b'
                     )

x = GlobalAveragePooling2D()(x)

x = Dropout(0.4)(x)

x = Dense(classes, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=[x,aux_1,aux_2], name="GoogleNet")

In [26]:
model.summary()

Model: "GoogleNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv_1_7x7/2 (Conv2D)          (None, 64, 64, 64)   9472        ['input_5[0][0]']                
                                                                                                  
 max_pool_1_3x3/2 (MaxPooling2D  (None, 32, 32, 64)  0           ['conv_1_7x7/2[0][0]']           
 )                                                                                                
                                                                                          

# Training and Evaluating

In [27]:
#Compile the model
model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy', 'sparse_categorical_crossentropy', 'sparse_categorical_crossentropy'], metrics=['accuracy'])

# Define the learning rate reduction callback
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',   
    factor=0.1,            
    patience=5,           
    min_lr=1e-6            
)
# Define the model checkpoint callback to save the best model
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Define the early stopping callback to stop training if validation loss does not improve
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with the learning rate reduction callback
history = model.fit(X_train, [y_train, y_train, y_train],validation_data=(X_val,[y_val, y_val, y_val]),epochs=10, batch_size=32,callbacks=[lr_callback,checkpoint_callback,early_stopping_callback])

# Evaluate the model
best_model = tf.keras.models.load_model('best_model.h5')
loss, accuracy = best_model.evaluate(X_test, y_test)

print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

Epoch 1/10
128/128 [==============================] - 160s 1s/step - loss: 3.2285 - output_loss: 1.0864 - auxilliary_output_1_loss: 1.0666 - auxilliary_output_2_loss: 1.0755 - output_accuracy: 0.4785 - auxilliary_output_1_accuracy: 0.4832 - auxilliary_output_2_accuracy: 0.4702 - val_loss: 3.0867 - val_output_loss: 1.0301 - val_auxilliary_output_1_loss: 1.0267 - val_auxilliary_output_2_loss: 1.0299 - val_output_accuracy: 0.5117 - val_auxilliary_output_1_accuracy: 0.5117 - val_auxilliary_output_2_accuracy: 0.5117 - lr: 0.0010
Epoch 2/10
128/128 [==============================] - 146s 1s/step - loss: 3.1572 - output_loss: 1.0468 - auxilliary_output_1_loss: 1.0569 - auxilliary_output_2_loss: 1.0535 - output_accuracy: 0.4968 - auxilliary_output_1_accuracy: 0.4805 - auxilliary_output_2_accuracy: 0.4883 - val_loss: 3.0561 - val_output_loss: 1.0207 - val_auxilliary_output_1_loss: 1.0166 - val_auxilliary_output_2_loss: 1.0188 - val_output_accuracy: 0.5117 - val_auxilliary_output_1_accuracy: 0.5

ValueError: too many values to unpack (expected 2)

In [32]:
best_model.evaluate(X_test, [y_test,y_test,y_test])

40/40 [==============================] - 9s 223ms/step - loss: 2.4763 - output_loss: 0.8296 - auxilliary_output_1_loss: 0.8167 - auxilliary_output_2_loss: 0.8301 - output_accuracy: 0.6117 - auxilliary_output_1_accuracy: 0.6297 - auxilliary_output_2_accuracy: 0.6086


[2.4763336181640625,
 0.8295881152153015,
 0.8166508674621582,
 0.8300944566726685,
 0.6117187738418579,
 0.629687488079071,
 0.608593761920929]

In [42]:
# Make predictions on new images
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes=predicted_classes[1,:]

# Generate the classification report
class_names = list(['Mild_Demented','Moderate_Demented','Non-Demented','Very_Mild_Demented'])
report = classification_report(y_test, predicted_classes, target_names=class_names)
print("Classification Report:\n", report)

# Generate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

40/40 [==============================] - 9s 235ms/step


ValueError: Found input variables with inconsistent numbers of samples: [1280, 4]

In [43]:
predicted_classes

array([626, 960, 169, 265])